In [32]:
import pandas as pd
import numpy as np
import pickle

In [2]:
close_returns_raw = pd.read_csv('dataset/close_returns.csv', index_col = 0, parse_dates = True)
high_prices_raw = pd.read_csv('dataset/high_prices.csv', index_col = 0, parse_dates = True)
low_prices_raw = pd.read_csv('dataset/low_prices.csv', index_col = 0, parse_dates = True)
volume_raw = pd.read_csv('dataset/volume.csv', index_col = 0, parse_dates = True)
signals_A_raw = pd.read_csv('dataset/signals_A.csv', index_col = 0, parse_dates = True)
signals_B_raw = pd.read_csv('dataset/signals_B.csv', index_col = 0, parse_dates = True)

In [3]:
close_returns_raw.head()

,A.0,A.1,A.2,A.3,A.4,A.5,A.6,A.7,A.8,A.9,...,A.990,A.991,A.992,A.993,A.994,A.995,A.996,A.997,A.998,A.999
time,,,,,,,,,,,,,,,,,,,,,
2017-01-03 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03 09:45:00,-0.010530,0.002627,-0.005865,-0.002815,-0.006875,0.004111,-0.007562,0.034228,-0.016570,-0.023604,...,0.002529,-0.005074,0.000942,0.005411,NaN,-0.002108,0.006905,-0.004139,0.006602,0.002468
2017-01-03 10:00:00,-0.005218,-0.003813,0.006697,-0.006235,-0.001562,0.030088,0.017317,-0.014377,0.002730,-0.003007,...,-0.001575,NaN,0.019034,-0.003286,NaN,-0.009330,NaN,0.004126,0.001973,0.023038
2017-01-03 10:15:00,0.004456,0.007341,0.006147,-0.001772,-0.001146,-0.018533,-0.015283,0.028084,-0.015930,0.001107,...,0.001118,NaN,-0.003259,-0.004713,NaN,-0.000681,NaN,0.008012,0.006369,-0.019062
2017-01-03 10:30:00,-0.001799,-0.010040,0.008337,-0.015490,0.000895,-0.022239,0.011906,NaN,0.034934,-0.008240,...,0.005739,NaN,-0.002320,-0.003620,NaN,-0.001107,0.007323,-0.018825,-0.004333,0.000031


In [16]:
#def train_test_split_and_fillna(dataframe):
#    # Fill forward 
#    train_data = dataframe[:4360]
#    test_data = dataframe[4360:]   

'2017-08-31 15:45:00'

In [4]:
median_volume_each_stock = volume_raw.median(axis = 0)
volume_normalised = volume_raw.apply(lambda x: x / median_volume_each_stock, axis=1)

In [5]:
volatility = (high_prices_raw - low_prices_raw) / low_prices_raw

In [6]:
def intra_day_forward_fillna_max_filling_1_hour_gap(dataframe):
    return dataframe.groupby(dataframe.index.dayofyear).apply(
        lambda x: x.fillna(method = "ffill", axis = 0, limit = 4))

In [7]:
volume_normalised_filled = intra_day_forward_fillna_max_filling_1_hour_gap(volume_normalised)
volatility_filled = intra_day_forward_fillna_max_filling_1_hour_gap(volatility)
signal_A_raw_filled = intra_day_forward_fillna_max_filling_1_hour_gap(signals_A_raw)
signal_B_raw_filled = intra_day_forward_fillna_max_filling_1_hour_gap(signals_B_raw)

In [8]:
close_returns_raw_filled = close_returns_raw.groupby(close_returns_raw.index.dayofyear).apply(
    lambda x: x.fillna(value = 0.0, axis = 0)) 
# Different from "ffill", if use "limit" here, it would specify maximum 
# total number of 0 that can be used to pad each col. Not what we want.

In [9]:
list_of_df_each_for_one_stock = [pd.concat([close_returns_raw_filled.iloc[:, a],
                                            volume_normalised_filled.iloc[:, a],
                                            volatility_filled.iloc[:, a],
                                            signal_A_raw_filled.iloc[:, a],
                                            signal_B_raw_filled.iloc[:, a]], axis = 1)
                                 for a in range(1000)] # Total 1000 stocks

In [10]:
for e in list_of_df_each_for_one_stock:
    e.columns = ["return", "volume", "volatility", "signal_A", "signal_B"]               

In [11]:
list_of_df_each_for_one_stock[0].head()

,return,volume,volatility,signal_A,signal_B
time,,,,,
2017-01-03 09:30:00,0.000000,1.357828,0.011823,NaN,NaN
2017-01-03 09:45:00,-0.010530,0.840949,0.003595,-0.012937,NaN
2017-01-03 10:00:00,-0.005218,1.191392,0.005506,-0.004742,NaN
2017-01-03 10:15:00,0.004456,0.554745,0.004171,-0.001138,NaN
2017-01-03 10:30:00,-0.001799,0.484170,0.004360,0.003315,NaN


In [12]:
list_of_df_each_for_one_stock[0].tail()

,return,volume,volatility,signal_A,signal_B
time,,,,,
2017-12-29 14:45:00,-0.014195,0.389359,0.000930,-0.014027,0.678396
2017-12-29 15:00:00,-0.005576,0.359596,0.001627,-0.006361,0.653249
2017-12-29 15:15:00,0.006572,0.628543,0.002169,0.005638,1.139014
2017-12-29 15:30:00,-0.005632,1.039431,0.001703,-0.003321,1.790983
2017-12-29 15:45:00,0.011733,5.173711,0.002170,0.011734,6.585453


In [13]:
# list_of_df_each_for_one_stock[0].groupby(list_of_df_each_for_one_stock[0].index.dayofyear)
# [test.get_group(x) for x in test.groups]
# [e for e in test1 if len(e.index) < 26]

In [14]:
list_of_df_by_stock_by_day_grouped = [x.groupby(x.index.dayofyear) 
                                      for x in list_of_df_each_for_one_stock]

In [15]:
list_of_df_by_stock_by_day = [x.get_group(a) 
                              for x in list_of_df_by_stock_by_day_grouped 
                              for a in x.groups]

In [16]:
list_of_df_by_stock_by_day[0].tail()

,return,volume,volatility,signal_A,signal_B
time,,,,,
2017-01-03 14:45:00,0.002773,1.267860,0.001912,0.002390,1.559230
2017-01-03 15:00:00,-0.006562,0.902536,0.002672,-0.008265,1.130021
2017-01-03 15:15:00,0.001064,0.785697,0.002004,0.003739,0.969713
2017-01-03 15:30:00,-0.008516,1.212756,0.002479,-0.010592,1.432389
2017-01-03 15:45:00,0.003238,4.798810,0.005139,-0.000370,4.572557


In [17]:
window_size = 13

def get_list_of_window_sized_df(df):
    return [df.iloc[i: i + window_size] for i in range(len(df.index) - window_size + 1)]

In [18]:
list_of_df_by_stock_by_window = [get_list_of_window_sized_df(x) 
                                 for x in list_of_df_by_stock_by_day]

In [19]:
list_of_df_by_stock_by_window_flatten = [window 
                                         for by_stock_by_day in list_of_df_by_stock_by_window
                                         for window in by_stock_by_day]

In [20]:
list_of_df_by_stock_by_window_flatten[0]

,return,volume,volatility,signal_A,signal_B
time,,,,,
2017-01-03 09:30:00,0.000000,1.357828,0.011823,NaN,NaN
2017-01-03 09:45:00,-0.010530,0.840949,0.003595,-0.012937,NaN
2017-01-03 10:00:00,-0.005218,1.191392,0.005506,-0.004742,NaN
2017-01-03 10:15:00,0.004456,0.554745,0.004171,-0.001138,NaN
2017-01-03 10:30:00,-0.001799,0.484170,0.004360,0.003315,NaN
2017-01-03 10:45:00,-0.001767,0.742527,0.002465,-0.004975,NaN
2017-01-03 11:00:00,0.000046,1.119380,0.003421,0.002896,NaN
2017-01-03 11:15:00,0.004687,0.694956,0.001901,0.003925,NaN
2017-01-03 11:30:00,-0.003899,0.841778,0.003386,-0.004027,NaN


In [30]:
# Use window.reset_index().unstack().to_frame().T will take 5 hours
list_of_df_by_stock_by_window_flatten_reindexed_unstacked = \
    [window.reset_index().unstack()
     for window in list_of_df_by_stock_by_window_flatten]

In [36]:
with open('list_of_df_by_stock_by_window_flatten_reindexed_unstacked.pickle', 'wb') as handle:
    pickle.dump(list_of_df_by_stock_by_window_flatten_reindexed_unstacked, handle, 
                protocol = pickle.HIGHEST_PROTOCOL)

#with open('list_of_df_by_stock_by_window_flatten_reindexed_unstacked.pickle', 'rb') as handle:
#    list_of_df_by_stock_by_window_flatten_reindexed_unstacked = pickle.load(handle)

KeyboardInterrupt: 

In [ ]:
tidy_table = pd.concat(list_of_df_by_stock_by_window_flatten_reindexed_unstacked, axis=1).T

In [ ]:
tidy_table.columns = [col[0] + '_' + str(col[1]) for col in tidy_table.columns.values]

In [ ]:
tidy_table = tidy_table.drop(['time_0', 'time_1', 'time_2', 'time_3', 'time_4', 'time_5', 'time_6',
                              'time_7', 'time_8', 'time_9', 'time_10', 'time_11'], axis = 1)

In [259]:
test = list_of_df_by_stock_by_window_flatten[0]

In [260]:
test.reset_index()

,time,return,volume,volatility,signal_A,signal_B
0,2017-01-03 09:30:00,0.000000,1.357828,0.011823,NaN,NaN
1,2017-01-03 09:45:00,-0.010530,0.840949,0.003595,-0.012937,NaN
2,2017-01-03 10:00:00,-0.005218,1.191392,0.005506,-0.004742,NaN
3,2017-01-03 10:15:00,0.004456,0.554745,0.004171,-0.001138,NaN
4,2017-01-03 10:30:00,-0.001799,0.484170,0.004360,0.003315,NaN
5,2017-01-03 10:45:00,-0.001767,0.742527,0.002465,-0.004975,NaN
6,2017-01-03 11:00:00,0.000046,1.119380,0.003421,0.002896,NaN
7,2017-01-03 11:15:00,0.004687,0.694956,0.001901,0.003925,NaN
8,2017-01-03 11:30:00,-0.003899,0.841778,0.003386,-0.004027,NaN
9,2017-01-03 11:45:00,-0.003412,1.025858,0.003048,-0.004419,1.158692


In [261]:
test.reset_index().unstack().to_frame()

0
time       0   2017-01-03 09:30:00
           1   2017-01-03 09:45:00
           2   2017-01-03 10:00:00
           3   2017-01-03 10:15:00
           4   2017-01-03 10:30:00
           5   2017-01-03 10:45:00
           6   2017-01-03 11:00:00
           7   2017-01-03 11:15:00
           8   2017-01-03 11:30:00
           9   2017-01-03 11:45:00
           10  2017-01-03 12:00:00
           11  2017-01-03 12:15:00
           12  2017-01-03 12:30:00
return     0                     0
           1            -0.0105299
           2           -0.00521836
           3            0.00445552
           4           -0.00179895
           5           -0.00176673
           6             4.567e-05
           7            0.00468657
           8           -0.00389929
           9           -0.00341166
           10            0.0107627
           11           -0.0163678
           12         -0.000866937
volume     0               1.35783
           1              0.840949
           2               1.19139
           3              0.554745
...                            ...
volatility 9            0.00304819
           10           0.00362112
           11           0.00247337
           12           0.00323935
signal_A   0                   NaN
           1            -0.0129373
           2           -0.00474177
           3             -0.001138
           4            0.00331483
           5           -0.00497514
           6            0.00289551
           7            0.00392517
           8           -0.00402671
           9            -0.0044192
           10            0.0101862
           11           -0.0158627
           12         -0.000524834
signal_B   0                   NaN
           1                   NaN
           2                   NaN
           3                   NaN
           4                   NaN
           5                   NaN
           6                   NaN
           7                   NaN
           8                   NaN
           9               1.15869
           10             0.773829
           11              0.75454
           12              1.99192

[78 rows x 1 columns]

In [250]:
test = list_of_df_by_stock_by_window[0][0]

In [251]:
test.unstack().to_frame().T

return                                          \
time 2017-01-03 09:30:00 2017-01-03 09:45:00 2017-01-03 10:00:00   
0                    0.0            -0.01053           -0.005218   

                                                                  \
time 2017-01-03 10:15:00 2017-01-03 10:30:00 2017-01-03 10:45:00   
0               0.004456           -0.001799           -0.001767   

                                                                  \
time 2017-01-03 11:00:00 2017-01-03 11:15:00 2017-01-03 11:30:00   
0               0.000046            0.004687           -0.003899   

                                 ...                    signal_B  \
time 2017-01-03 11:45:00         ...         2017-01-03 10:15:00   
0              -0.003412         ...                         NaN   

                                                                  \
time 2017-01-03 10:30:00 2017-01-03 10:45:00 2017-01-03 11:00:00   
0                    NaN                 NaN                 NaN   

                                                                  \
time 2017-01-03 11:15:00 2017-01-03 11:30:00 2017-01-03 11:45:00   
0                    NaN                 NaN            1.158692   

                                                                  
time 2017-01-03 12:00:00 2017-01-03 12:15:00 2017-01-03 12:30:00  
0               0.773829             0.75454            1.991922  

[1 rows x 65 columns]

In [236]:
test4 = test3[0:2]

In [237]:
test5 = [make_more(e) for e in test4]

In [238]:
test5

[[                       return    volume  volatility  signal_A  signal_B
  time                                                                   
  2017-01-03 09:30:00  0.000000  1.357828    0.011823       NaN       NaN
  2017-01-03 09:45:00 -0.010530  0.840949    0.003595 -0.012937       NaN
  2017-01-03 10:00:00 -0.005218  1.191392    0.005506 -0.004742       NaN
  2017-01-03 10:15:00  0.004456  0.554745    0.004171 -0.001138       NaN
  2017-01-03 10:30:00 -0.001799  0.484170    0.004360  0.003315       NaN
  2017-01-03 10:45:00 -0.001767  0.742527    0.002465 -0.004975       NaN
  2017-01-03 11:00:00  0.000046  1.119380    0.003421  0.002896       NaN
  2017-01-03 11:15:00  0.004687  0.694956    0.001901  0.003925       NaN
  2017-01-03 11:30:00 -0.003899  0.841778    0.003386 -0.004027       NaN
  2017-01-03 11:45:00 -0.003412  1.025858    0.003048 -0.004419  1.158692
  2017-01-03 12:00:00  0.010763  0.669963    0.003621  0.010186  0.773829
  2017-01-03 12:15:00 -0.016368  0.639

In [235]:
window_size = 12
# The first window is from row 0 to 3, i.e. iloc[0: 0 + window_size - 1]
# The last window is from row len(x.index) - window_size + 1 to len(x.index)
# len(x.index) - window_size + 1 is 
#test.apply(lambda x: x.iloc[i: i + k] for i in range(len(x.index) - k + 1))
#test2 = test.apply(lambda x: x.iloc[i: i + window_size] for i in range(len(x.index) - window_size + 1))
#.apply(lambda x: pd.concat([x.iloc[i: i + k] for i in range(len(x.index) - k + 1)]))])
def make_more(df):
    return [df.iloc[i: i + window_size] for i in range(len(df.index) - window_size + 1)]

#test2 = test.apply(lambda x: make_more(df))

In [234]:
test4[0]

,return,volume,volatility,signal_A,signal_B
time,,,,,
2017-01-03 09:30:00,0.000000,1.357828,0.011823,NaN,NaN
2017-01-03 09:45:00,-0.010530,0.840949,0.003595,-0.012937,NaN
2017-01-03 10:00:00,-0.005218,1.191392,0.005506,-0.004742,NaN
2017-01-03 10:15:00,0.004456,0.554745,0.004171,-0.001138,NaN
2017-01-03 10:30:00,-0.001799,0.484170,0.004360,0.003315,NaN
2017-01-03 10:45:00,-0.001767,0.742527,0.002465,-0.004975,NaN
2017-01-03 11:00:00,0.000046,1.119380,0.003421,0.002896,NaN
2017-01-03 11:15:00,0.004687,0.694956,0.001901,0.003925,NaN
2017-01-03 11:30:00,-0.003899,0.841778,0.003386,-0.004027,NaN
